In [8]:
import torch
import requests
import json
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [6]:
def get_data(i):
    visual_news_data = json.load(open("../../../datasets/visualnews/origin/data.json"))
    visual_news_data_mapping = {ann["id"]: ann for ann in visual_news_data}

    data = json.load(open("../../../news_clippings/news_clippings/data/merged_balanced/test.json"))
    annotations = data["annotations"]
    ann_true = annotations[i]

    caption = visual_news_data_mapping[ann_true["id"]]["caption"]
    image_path = visual_news_data_mapping[ann_true["image_id"]]["image_path"]
    image_path = "../../../datasets/visualnews/origin/"+image_path[2:]
    image = Image.open(image_path)
    #print("DATA SAMPLE")
    #print("Caption: ", caption)
    #print("Misinformation (Ground Truth): {}".format(ann_true["falsified"]))
    return image, caption, image_path, ann_true

In [5]:
torch_dtype = torch.float16

model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32",
    device_map="auto",
    torch_dtype=torch_dtype,
)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [47]:
image, caption, image_path, annotation = get_data(4)

In [48]:
prompt1 = "The caption: {}, matches the image".format(caption)
prompt2 = "The caption: {}, does not match the image".format(caption)

In [49]:
inputs = processor(text=[prompt1, prompt2], images=image, return_tensors="pt", padding=True)

In [50]:
outputs = model(**inputs)

In [51]:
probs = outputs.logits_per_image.softmax(dim=1)

In [52]:
results = [False, True]

In [53]:
results[int(torch.argmax(probs))]

False